# **Library**

In [1]:
!pip install opencv-python numpy paho-mqtt

In [2]:
import cv2
import numpy as np
import json, time, os
import paho.mqtt.client as mqtt

def show_image(img, title="Output"):
    cv2.imshow(title, img)

# **Config**

## Camera

In [3]:
# === Camera / algorithm config (tuned to accept elongated/curved dough) ===
PIXELS_PER_CM = 19.23

# Dough HSV heuristics (white/desaturated + bright)
S_MAX_WHITE = 40
V_MIN_WHITE = 170   # lowered from 190 to be more tolerant to lighting

# Background hue auto-veto (robust to local background color)
BG_S_MIN = 45
BG_H_BAND = 15

# Area / shape thresholds
MIN_DOUGH_AREA = 800      # lowered from 1200 to allow smaller/shorter pieces
MAX_AREA_FRAC = 0.45      # max fraction of image to consider (avoid full-frame blobs)
BOX_FILL_MIN = 0.20       # allow looser box-fill for curved shapes (was 0.30)
MIN_CONTAMINANT_AREA = 60

# Morphology / processing
KERNEL_SIZE = 5
BH_KERNEL = 9             # black-hat kernel size (odd)
K_SIGMA = 3.0

# Interior margin (remove border shading/noise)
PIXEL_MARGIN_MIN = 8
MARGIN_FRAC = 0.4         # multiplier of PIXELS_PER_CM for margin_px

# --- New: color/intensity checks to reduce false positives ---
DELTA_AB_MIN = 10.0
DELTA_L_MIN = 18.0
BH_MEAN_FACTOR = 1.0
BH_MEAN_MIN = 10.0
SOLIDITY_MIN = 0.55

# --- Border and containment tightening (relaxed a bit) ---
BORDER_MARGIN_PX = 12
MIN_INSIDE_RATIO = 0.80   # relaxed from 0.90

# --- Candidate geometry/color constraints (for dough) ---
CANDIDATE_SOLIDITY_MIN = 0.60   # relaxed from 0.85 (curved dough can give ~0.6)
MIN_ASPECT_RATIO = 0.25        # allow longer shapes (was 0.35)

# Existing Combine constants (kept for compatibility)
LOWER_DOUGH_COLOR = np.array([0, 0, 150])
UPPER_DOUGH_COLOR = np.array([179, 55, 255])
LOWER_BLUE_BG = np.array([100, 50, 50])
UPPER_BLUE_BG = np.array([130, 255, 255])

# sanity
kernel5 = np.ones((5, 5), np.uint8)

## MQTT

In [4]:
MQTT_HOST = 'test.mosquitto.org'
MQTT_PORT = 1883
MQTT_TOPIC = 'CPRAM/1/cam'
PUBLISH_INTERVAL_SEC = 0.1

client = mqtt.Client(client_id=f"dough-cam-{np.random.randint(0,1e9)}")
client.connect(MQTT_HOST, MQTT_PORT, 60)
client.loop_start()

def publish_measure(length_cm, contaminant_status):
    payload = {"module": "cam", "data": f"{int(round(length_cm))},{int(contaminant_status)}"}
    client.publish(MQTT_TOPIC, json.dumps(payload), qos=0)

/tmp/ipykernel_587344/1303021691.py:6: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client(client_id=f"dough-cam-{np.random.randint(0,1e9)}")


## Opencv

In [5]:
# Helpers: morphology / border checks (from test_detect)
def build_safe_zone(shape, margin_px):
    h, w = shape
    sz = np.zeros((h, w), np.uint8)
    cv2.rectangle(sz, (margin_px, margin_px), (w - margin_px - 1, h - margin_px - 1), 255, -1)
    return sz

def touches_image_border_mask(mask_bin, erode_px=2):
    k = np.ones((3,3), np.uint8)
    m = cv2.erode(mask_bin, k, iterations=max(0, erode_px))
    h, w = m.shape
    return bool(np.any(m[0,:]) or np.any(m[-1,:]) or np.any(m[:,0]) or np.any(m[:,-1]))

def border_checks_for_contour(contour, mask_shape, margin_px):
    h, w = mask_shape
    m = np.zeros(mask_shape, np.uint8)
    cv2.drawContours(m, [contour], -1, 255, -1)
    x, y, bw, bh = cv2.boundingRect(contour)
    touch_by_bbox = (x <= margin_px) or (y <= margin_px) or (x + bw >= w - margin_px) or (y + bh >= h - margin_px)
    erode_px = max(1, min(6, margin_px // 3))
    touch_by_mask = touches_image_border_mask(m, erode_px=erode_px)
    safe = build_safe_zone(mask_shape, margin_px)
    inside = cv2.bitwise_and(m, safe)
    area_all = cv2.countNonZero(m) + 1e-6
    inside_ratio = cv2.countNonZero(inside) / area_all
    touch = bool(touch_by_mask or touch_by_bbox)
    return inside_ratio, touch

# **Process Fram**

In [6]:
def process_frame(frame):
    output_frame = frame.copy()
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    H, S, V = cv2.split(hsv)

    # 1) Candidate dough mask: low saturation and high value (white-ish)
    white_mask = ((S < S_MAX_WHITE) & (V > V_MIN_WHITE)).astype(np.uint8) * 255

    # 2) Estimate background hue from border and veto similar hue
    h_img, w_img = white_mask.shape
    edge = max(10, int(min(h_img, w_img) * 0.06))
    bg_mask = np.zeros_like(white_mask)
    bg_mask[:edge, :] = 255
    bg_mask[-edge:, :] = 255
    bg_mask[:, :edge] = 255
    bg_mask[:, -edge:] = 255
    bg_mask = cv2.erode(bg_mask, np.ones((9,9), np.uint8))
    bg_h_vals = H[(bg_mask > 0) & (S > BG_S_MIN)]
    bg_h = int(np.median(bg_h_vals)) if bg_h_vals.size else 0
    dh = np.abs(H.astype(int) - int(bg_h))
    dh = np.minimum(dh, 180 - dh)
    bg_color_mask = ((dh <= BG_H_BAND) & (S > BG_S_MIN)).astype(np.uint8) * 255

    # combine and cleanup
    dough_mask = cv2.bitwise_and(white_mask, cv2.bitwise_not(bg_color_mask))
    kernel = np.ones((KERNEL_SIZE, KERNEL_SIZE), np.uint8)
    dough_mask_clean = cv2.morphologyEx(dough_mask, cv2.MORPH_CLOSE, kernel)
    dough_mask_clean = cv2.morphologyEx(dough_mask_clean, cv2.MORPH_OPEN, kernel)

    # find contours and choose best dough candidate (with stricter checks)
    contours, _ = cv2.findContours(dough_mask_clean, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    h_img, w_img = dough_mask_clean.shape
    img_area = h_img * w_img
    found_dough = False
    c_dough = None
    reason = 'no contour'
    chosen_reason = reason
    best_score = -1.0

    for c in contours:
        area = cv2.contourArea(c)
        if area < MIN_DOUGH_AREA:
            reason = 'too small'; continue
        if area > MAX_AREA_FRAC * img_area:
            reason = 'too large'; continue
        x, y, w, h = cv2.boundingRect(c)
        box_fill = area / (w * h + 1e-6)
        if box_fill < BOX_FILL_MIN:
            reason = f'box_fill low ({box_fill:.2f})'; continue

        inside_ratio, touch = border_checks_for_contour(c, dough_mask_clean.shape, BORDER_MARGIN_PX)
        if touch:
            reason = 'touches border (bbox/mask)'; continue
        if inside_ratio < MIN_INSIDE_RATIO:
            reason = f'inside_ratio low ({inside_ratio:.2f})'; continue

        # color checks inside contour
        mask_c = np.zeros_like(dough_mask_clean, dtype=np.uint8)
        cv2.drawContours(mask_c, [c], -1, 255, thickness=cv2.FILLED)
        cnt_pixels = np.count_nonzero(mask_c)
        if cnt_pixels == 0:
            reason = 'empty mask'; continue
        mean_S = float(np.mean(S[mask_c > 0]))
        mean_V = float(np.mean(V[mask_c > 0]))
        if not (mean_S <= S_MAX_WHITE and mean_V >= V_MIN_WHITE):
            reason = f'color mismatch (S={mean_S:.1f},V={mean_V:.1f})'; continue

        # geometry: avoid long thin strips and require solidity
        aspect = min(w, h) / (max(w, h) + 1e-6)
        hull = cv2.convexHull(c)
        hull_area = cv2.contourArea(hull) + 1e-6
        solidity = area / hull_area
        if aspect < MIN_ASPECT_RATIO:
            reason = f'aspect low ({aspect:.2f})'; continue
        if solidity < CANDIDATE_SOLIDITY_MIN:
            reason = f'solidity low ({solidity:.2f})'; continue

        # veto if matches background hue
        mean_H = float(np.mean(H[mask_c > 0]))
        dhc = abs(int(mean_H) - int(bg_h))
        dhc = min(dhc, 180 - dhc)
        if (mean_S > BG_S_MIN) and (dhc <= BG_H_BAND):
            reason = f'bg hue match (dh={dhc:.0f}, mean_S={mean_S:.0f})'; continue

        score = area
        if score > best_score:
            best_score = score
            c_dough = c
            found_dough = True
            chosen_reason = 'ok'

    # If no dough found -> return zeros as requested
    if not found_dough or c_dough is None:
        # annotate for debugging
        cv2.putText(output_frame, f'NO DOUGH ({reason})', (20,30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,255), 2)
        return False, 0.0, 0, output_frame

    # compute length as before (perimeter/2 → approximate length) and draw dough
    per = cv2.arcLength(c_dough, True)
    length_px = per / 2.0
    length_cm = length_px / PIXELS_PER_CM
    cv2.drawContours(output_frame, [c_dough], -1, (0, 255, 0), 2)

    # --- contaminant detection (same approach as test_detect) ---
    dough_filled = np.zeros_like(dough_mask_clean)
    cv2.drawContours(dough_filled, [c_dough], -1, 255, thickness=cv2.FILLED)
    margin_px = max(PIXEL_MARGIN_MIN, int(MARGIN_FRAC * PIXELS_PER_CM))
    dist = cv2.distanceTransform(dough_filled, cv2.DIST_L2, 3)
    interior = np.uint8((dist > margin_px) * 255)

    lab = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
    L = lab[:, :, 0]
    L_blur = cv2.GaussianBlur(L, (5, 5), 0)
    kernel_bh = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (BH_KERNEL, BH_KERNEL))
    blackhat = cv2.morphologyEx(L_blur, cv2.MORPH_BLACKHAT, kernel_bh)
    bh_in = cv2.bitwise_and(blackhat, blackhat, mask=interior)

    vals = bh_in[interior > 0]
    mu, sigma = (float(vals.mean()), float(vals.std())) if vals.size > 0 else (0.0, 0.0)
    t = mu + K_SIGMA * sigma
    _, spots_raw = cv2.threshold(bh_in, t, 255, cv2.THRESH_BINARY)
    spots = cv2.morphologyEx(spots_raw, cv2.MORPH_OPEN, np.ones((3,3), np.uint8), iterations=1)
    spots = cv2.morphologyEx(spots, cv2.MORPH_CLOSE, np.ones((5,5), np.uint8), iterations=1)

    contours_contam, _ = cv2.findContours(spots, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contaminant_status = 0

    lab_ab = lab[..., 1:3].astype(np.float32)
    mask_core = (interior > 0)
    if np.count_nonzero(mask_core) > 0:
        ref_ab = np.median(lab_ab[mask_core], axis=0)
        ref_L = float(np.median(L[mask_core]))
    else:
        ref_ab = np.array([0.0, 0.0], dtype=np.float32)
        ref_L = 128.0

    for cc in contours_contam:
        area_c = cv2.contourArea(cc)
        if area_c < MIN_CONTAMINANT_AREA:
            continue
        x, y, w, h = cv2.boundingRect(cc)
        per_c = cv2.arcLength(cc, True) + 1e-6
        circ = 4 * np.pi * area_c / (per_c * per_c)
        extent = area_c / (w * h + 1e-6)
        cx, cy = np.mean(cc.reshape(-1, 2), axis=0).astype(int)
        if dist[cy, cx] < margin_px:
            continue
        if extent < 0.12 and circ < 0.18:
            continue
        hull = cv2.convexHull(cc)
        hull_area = cv2.contourArea(hull) + 1e-6
        solidity = area_c / hull_area
        if solidity < SOLIDITY_MIN:
            continue
        # per-spot color checks
        mask_c = np.zeros_like(spots, dtype=np.uint8)
        cv2.drawContours(mask_c, [cc], -1, 255, thickness=cv2.FILLED)
        spot_pixels = (mask_c > 0)
        if np.count_nonzero(spot_pixels) == 0:
            continue
        mean_ab_spot = np.mean(lab_ab[spot_pixels], axis=0)
        delta_ab = float(np.linalg.norm(mean_ab_spot - ref_ab))
        mean_L_spot = float(np.mean(L[spot_pixels]))
        delta_L = float(ref_L - mean_L_spot)
        mean_dark = float(cv2.mean(bh_in, mask=mask_c)[0])
        intensity_threshold = max(BH_MEAN_MIN, mu + BH_MEAN_FACTOR * sigma)
        cues = 0
        if delta_ab >= DELTA_AB_MIN:
            cues += 1
        if delta_L >= DELTA_L_MIN:
            cues += 1
        if mean_dark >= intensity_threshold:
            cues += 1
        if cues < 2:
            continue
        contaminant_status = 1
        cv2.rectangle(output_frame, (x,y), (x+w, y+h), (0,0,255), 2)
        cv2.putText(output_frame, f'Contam Δab={delta_ab:.1f},ΔL={delta_L:.1f}', (x, y-8), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 2)
        break

    # annotate length and status
    x_b, y_b, w_b, h_b = cv2.boundingRect(c_dough)
    cv2.putText(output_frame, f"{length_cm:.2f} cm", (x_b, y_b - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    if contaminant_status == 1:
        cv2.putText(output_frame, "Contaminant Found!", (x_b, y_b - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    return True, length_cm, contaminant_status, output_frame


# **Show**

In [ ]:
# ...existing show / capture loop (unchanged) ...
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Warning: ไม่สามารถเปิดกล้องได้ — พยายามใช้ไฟล์วิดีโอสำรองหรือโชว์ภาพตัวอย่าง")
    fallback_video = "sample.mp4"
    if os.path.exists(fallback_video):
        cap = cv2.VideoCapture(fallback_video)
        print(f"Opened fallback video: {fallback_video}")
    else:
        print("Fallback video not found. Running one-frame demo using a synthetic image.")
        demo_img = np.full((480, 640, 3), 200, dtype=np.uint8)
        found, length_cm, status, out = process_frame(demo_img)
        print(f"ความยาว (demo): {length_cm:.2f} cm, สิ่งแปลกปลอม: {status}")
        show_image(out, title="Demo Output")
        cv2.waitKey(0)
        client.loop_stop()
        client.disconnect()
        raise SystemExit("Demo complete — no camera or fallback video available.")

print("กด q เพื่อออกจากโปรแกรม")
last_pub_t = 0.0

while True:
    ret, frame = cap.read()
    if not ret:
        print("Stream ended or cannot read frame — exiting loop")
        break
    found, length_cm, status, out = process_frame(frame)
    print(f"ความยาว: {length_cm:.2f} cm, สิ่งแปลกปลอม: {status}")
    show_image(out)
    now = time.monotonic()
    if now - last_pub_t >= PUBLISH_INTERVAL_SEC:
        publish_measure(length_cm if found else 0.0, status if found else 0)
        last_pub_t = now
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
client.loop_stop()
client.disconnect()

กด q เพื่อออกจากโปรแกรม
ความยาว: 3.48 cm, สิ่งแปลกปลอม: 0
ความยาว: 3.48 cm, สิ่งแปลกปลอม: 0
ความยาว: 3.48 cm, สิ่งแปลกปลอม: 0
ความยาว: 3.51 cm, สิ่งแปลกปลอม: 0
ความยาว: 3.53 cm, สิ่งแปลกปลอม: 0
ความยาว: 3.53 cm, สิ่งแปลกปลอม: 0
ความยาว: 3.57 cm, สิ่งแปลกปลอม: 0
ความยาว: 3.48 cm, สิ่งแปลกปลอม: 0
ความยาว: 3.51 cm, สิ่งแปลกปลอม: 0
ความยาว: 3.53 cm, สิ่งแปลกปลอม: 0
ความยาว: 3.53 cm, สิ่งแปลกปลอม: 0
ความยาว: 3.57 cm, สิ่งแปลกปลอม: 0
ความยาว: 3.57 cm, สิ่งแปลกปลอม: 0
ความยาว: 0.00 cm, สิ่งแปลกปลอม: 0
ความยาว: 0.00 cm, สิ่งแปลกปลอม: 0
ความยาว: 3.57 cm, สิ่งแปลกปลอม: 0
ความยาว: 0.00 cm, สิ่งแปลกปลอม: 0
ความยาว: 0.00 cm, สิ่งแปลกปลอม: 0
ความยาว: 0.00 cm, สิ่งแปลกปลอม: 0
ความยาว: 0.00 cm, สิ่งแปลกปลอม: 0
ความยาว: 0.00 cm, สิ่งแปลกปลอม: 0
ความยาว: 0.00 cm, สิ่งแปลกปลอม: 0
ความยาว: 0.00 cm, สิ่งแปลกปลอม: 0
ความยาว: 0.00 cm, สิ่งแปลกปลอม: 0
ความยาว: 0.00 cm, สิ่งแปลกปลอม: 0
ความยาว: 0.00 cm, สิ่งแปลกปลอม: 0
ความยาว: 0.00 cm, สิ่งแปลกปลอม: 0
ความยาว: 0.00 cm, สิ่งแปลกปลอม: 0
ความยาว: 0.00 cm, สิ่งแป

KeyboardInterrupt: 

: 